In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import os
import sys
import shutil
import math
import random
import heapq 
import time
import copy
import itertools  
from PIL import Image
from io import StringIO,BytesIO 
from scipy.spatial.distance import pdist
import cv2
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,roc_curve,accuracy_score,auc 
from functools import reduce
import wfdb#https://github.com/MIT-LCP/wfdb-python
from wfdb import processing
import faiss 
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
torch.cuda.set_device(3)
print (torch.cuda.current_device())

Loading faiss with AVX2 support.


3


In [2]:
#read train image with CV
train_dir = '/data/fjsdata/ECG/MIT-BIH/train' #the path of images
trN, trI, trY = [],[],[]
for iname in os.listdir(train_dir):
    if iname.endswith(".png"):
        try:
            image_path = os.path.join(train_dir, iname)
            itype = int(os.path.splitext(iname)[0].split("-")[1])
            img = cv2.resize(cv2.imread(image_path).astype(np.float32), (256, 256))#(500,300,3)->(256,256,3)
            trN.append(iname)
            trI.append(img)
            trY.append(itype)
        except:
            print(iname+":"+str(image_path))
        sys.stdout.write('\r{} / {} '.format(len(trY),20000))
        sys.stdout.flush()
print('The length of train set is %d'%len(trY))
#read test image with CV
test_dir = '/data/fjsdata/ECG/MIT-BIH/test' #the path of images
teN, teI, teY = [],[],[]
for iname in os.listdir(test_dir):
    if iname.endswith(".png"):
        try:
            image_path = os.path.join(test_dir, iname)
            itype = int(os.path.splitext(iname)[0].split("-")[1])
            img = cv2.resize(cv2.imread(image_path).astype(np.float32), (256, 256))#(500,300,3)->(256,256,3)
            teN.append(iname)
            teI.append(img)
            teY.append(itype)
        except:
            print(iname+":"+str(image_path))
        sys.stdout.write('\r{} / {} '.format(len(teY),20000))
        sys.stdout.flush()
print('The length of test set is %d'%len(teY))

20000 / 20000 The length of train set is 20000
20000 / 20000 The length of test set is 20000


In [3]:
class SpatialAttention(nn.Module):#spatial attention layer
    def __init__(self):
        super(SpatialAttention, self).__init__()

        self.conv1 = nn.Conv2d(2, 1, kernel_size=3, padding=1, bias=False)
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x = torch.cat([avg_out, max_out], dim=1)
        x = self.conv1(x)
        return self.sigmoid(x)
    
class ResBlock(nn.Module):
    def __init__(self, in_channels: int, out_channels: int, stride=1):
        super().__init__()
        self.net = nn.Sequential(
            nn.Conv2d(
                in_channels=in_channels, out_channels=out_channels,
                kernel_size=3, stride=stride, padding=1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
        )

        self.downsample_layer = None
        self.do_downsample = False
        if in_channels != out_channels or stride != 1:
            self.do_downsample = True
            self.downsample_layer = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, 3, stride, 1, bias=False),
                nn.BatchNorm2d(out_channels),
            )

        # initialize weights
        self.apply(self.init_weights)

    def forward(self, x):
        identity = x
        out = self.net(x)

        if self.do_downsample:
            identity = self.downsample_layer(x)

        return F.relu(out + identity, inplace=True) #resnet

    @staticmethod
    def init_weights(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)
            
class ATHNet(nn.Module):
    def __init__(self, hash_size: int, type_size: int):
        super().__init__()
        #resnet and maxpool
        self.net1 = nn.Sequential(#(3,256,256)->(16,128,128)
            ResBlock(in_channels=3, out_channels=16, stride=2), 
            nn.MaxPool2d(kernel_size=3, padding=1, stride=1)
        )
        
        #Attention (16,128,128)->(16,128,128)
        self.sa = SpatialAttention()
        
        #resnet and meanpool
        self.net2 =nn.Sequential( #(16,128,128)->(8,64,64)
            ResBlock(in_channels=16, out_channels=8, stride=2),
            nn.AvgPool2d(kernel_size=3, padding=1, stride=1)
        ) 
         
        #fully connected with conv (8,64,64)->(1,32,32)
        self.dense=ResBlock(in_channels=8, out_channels=1, stride=2)
        #fully connected (1,32,32)->class_size
        self.hashlayer = nn.Linear(1*32*32, hash_size)
        self.typelayer = nn.Linear(1*32*32, type_size)
    
        # initialize weights
        self.apply(self.init_weights)

    def forward(self, x):
        x = self.net1(x)
        x = self.sa(x)*x
        x = self.net2(x)
        x = self.dense(x)
        x = x.view(x.size(0),-1)
        x_hash = self.hashlayer(x)
        x_type = self.typelayer(x)
        return x_hash, x_type

    @staticmethod
    def init_weights(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.Linear):
            nn.init.xavier_normal_(m.weight)

class HashLossFunc(nn.Module):
    def __init__(self, margin=0.5, alpha=0.01):
        super(HashLossFunc, self).__init__()
        self.alpha = alpha #regularization
        self.margin = margin #margin threshold
        self.mse_loss = nn.MSELoss(reduction='none')
        self.l1_loss = nn.L1Loss(reduction='mean')
    
    def forward(self,h1,h2,y):    
        margin_val = self.margin * h1.shape[1]
        squared_loss = torch.mean(self.mse_loss(h1, h2), dim=1)
        # T1: 0.5 * (1 - y) * dist(x1, x2)
        positive_pair_loss = (0.5 * (1 - y) * squared_loss)
        mean_positive_pair_loss = torch.mean(positive_pair_loss)
        # T2: 0.5 * y * max(margin - dist(x1, x2), 0)
        zeros = torch.zeros_like(squared_loss)
        marginMat = margin_val * torch.ones_like(squared_loss)
        negative_pair_loss = 0.5 * y * torch.max(zeros, marginMat - squared_loss)
        mean_negative_pair_loss = torch.mean(negative_pair_loss)

        # T3: alpha(dst_l1(abs(x1), 1)) + dist_l1(abs(x2), 1)))
        mean_value_regularization = self.alpha * (
                self.l1_loss(torch.abs(h1), torch.ones_like(h1)) +
                self.l1_loss(torch.abs(h2), torch.ones_like(h2)))

        loss = mean_positive_pair_loss + mean_negative_pair_loss + mean_value_regularization
        return loss
    
#https://github.com/luyajie/triplet-deep-hash-pytorch#triplet-deep-hash-pytorch            
class TripletLoss(nn.Module):
    def __init__(self, margin=0.5):
        super(TripletLoss, self).__init__()
        self.margin = margin #margin threshold
        self.mse_loss = nn.MSELoss(reduction='none')
    
    def forward(self,H_q,H_p,H_n):    
        margin_val = self.margin * H_q.shape[1]
        squared_loss_pos = torch.mean(self.mse_loss(H_q, H_p), dim=1)
        squared_loss_neg = torch.mean(self.mse_loss(H_q, H_n), dim=1)
        zeros = torch.zeros_like(squared_loss_neg)
        loss  = torch.max(zeros, margin_val - squared_loss_neg + squared_loss_pos)
        return torch.mean(loss)

class FocalLoss(nn.Module):
    #Loss(x, class) = - \alpha (1-softmax(x)[class])^gamma \log(softmax(x)[class])
    def __init__(self, gamma=0, alpha=None, size_average=True):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.alpha = alpha
        if isinstance(alpha,(float,int)): self.alpha = torch.Tensor([alpha,1-alpha])
        if isinstance(alpha,list): self.alpha = torch.Tensor(alpha)
        self.size_average = size_average

    def forward(self, out, y):
        y = y.view(-1,1)
        logpt = F.log_softmax(out,dim=1)#default ,dim=1
        logpt = logpt.gather(1,y)# dim=1, index=y, max
        logpt = logpt.view(-1)
        pt = Variable(logpt.data.exp())

        if self.alpha is not None:
            if self.alpha.type()!=out.data.type():
                self.alpha = self.alpha.type_as(out.data)
            at = self.alpha.gather(0,y.data.view(-1))
            logpt = logpt * Variable(at)

        loss = -1 * (1-pt)**self.gamma * logpt
        if self.size_average: return loss.mean()
        else: return loss.sum()

class CircleLoss(nn.Module):
    def __init__(self, scale=32, margin=0.25, similarity='cos', **kwargs):
        super(CircleLoss, self).__init__()
        self.scale = scale
        self.margin = margin
        self.similarity = similarity

    def forward(self, feats, labels):
        assert feats.size(0) == labels.size(0), \
            f"feats.size(0): {feats.size(0)} is not equal to labels.size(0): {labels.size(0)}"
        batch_size = feats.size(0)
        if self.similarity == 'dot':
            sim_mat = torch.matmul(feats, torch.t(feats))
        elif self.similarity == 'cos':
            feats = F.normalize(feats)
            sim_mat = feats.mm(feats.t())
        else:
            raise ValueError('This similarity is not implemented.')
        loss = list()
        for i in range(batch_size):
            pos_index = labels == labels[i]
            pos_index[i] = 0
            neg_index = labels != labels[i]
            pos_pair_ = sim_mat[i][pos_index]
            neg_pair_ = sim_mat[i][neg_index]

            alpha_p = torch.relu(-pos_pair_ + 1 + self.margin)
            alpha_n = torch.relu(neg_pair_ + self.margin)
            margin_p = 1 - self.margin
            margin_n = self.margin
            loss_p = torch.sum(torch.exp(-self.scale * alpha_p * (pos_pair_ - margin_p)))
            loss_n = torch.sum(torch.exp(self.scale * alpha_n * (neg_pair_ - margin_n)))
            loss.append(torch.log(1 + loss_p * loss_n))

        loss = sum(loss) / batch_size
        return loss

#Generate image pairs for model
def onlineGenImgPairs():
    if (len(trY) % 2) == 0: spls = len(trY)
    else:  spls = len(trY)-1
    idx_sf = random.sample(range(0, spls),spls)
    trI1_sf, trI2_sf, trY1_sf, trY2_sf = [],[],[],[]
    flag = 0
    for i in idx_sf:
        if flag==0:
            trI1_sf.append(trI[i])
            trY1_sf.append(trY[i])
            flag =1
        else:
            trI2_sf.append(trI[i])
            trY2_sf.append(trY[i])
            flag =0
    trY_sf = np.where((np.array(trY1_sf)-np.array(trY2_sf))!=0,1,0)
    return np.array(trI1_sf),np.array(trI2_sf),trY_sf

#Generate image pairs for model
def onlineGenImgTriplets( ):
    idx_sf = []
    idx_0 = np.where( np.array(trY) == 0 ) #class 0
    idx_0 = list(idx_0[0])[0:4555]
    idx_sf.extend(idx_0)
    idx_1 = np.where( np.array(trY) == 1 ) #class 1
    idx_1 = list(idx_1[0])
    idx_sf.extend(idx_1)
    idx_2 = np.where( np.array(trY) == 2 ) #class 2
    idx_2 = list(idx_2[0])
    idx_sf.extend(idx_2)
    idx_3 = np.where( np.array(trY) == 3 ) #class 3
    idx_3 = list(idx_3[0])
    idx_sf.extend(idx_3)
    random.shuffle(idx_sf)   
    trQ_sf, trP_sf, trN_sf = [], [], []
    trQ_y, trP_y, trN_y = [], [], []
    for iQ in idx_sf:
        trQ_sf.append(trI[iQ])
        trQ_y.append(trY[iQ])
        if trY[iQ] == 0:
            idx_tmp = idx_0.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_0))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 1:
            idx_tmp = idx_1.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_1))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 2:
            idx_tmp = idx_2.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_2))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        elif trY[iQ] == 3:
            idx_tmp = idx_3.copy()
            idx_tmp.remove(iQ)
            iP =  random.sample(idx_tmp,1) #remove self,then get one positive sample
            trP_sf.append(trI[iP[0]])
            trP_y.append(trY[iP[0]])
            idx_sf_tmp = list(set(idx_sf) - set(idx_3))
            iN =  random.sample(idx_sf_tmp,1) #remove positive and get one negative sample
            trN_sf.append(trI[iN[0]])
            trN_y.append(trY[iN[0]])
        else: pass
        sys.stdout.write('\r{} / {} '.format(len(trQ_sf),len(idx_sf)))
        sys.stdout.flush()
    return np.array(trQ_sf),np.array(trP_sf),np.array(trN_sf), np.array(trQ_y), np.array(trP_y), np.array(trN_y)

In [39]:
#trQ_sf, trP_sf, trN_sf, trQ_y, trP_y, trN_y = onlineGenImgTriplets() #sample  triplet labels
assert (trQ_sf.shape==trP_sf.shape and trQ_sf.shape==trN_sf.shape)
assert (trQ_y.shape==trP_y.shape and trQ_y.shape==trN_y.shape)
assert (np.mean(np.where((np.array(trQ_y)-np.array(trP_y))!=0,1,0))==0.0)
assert (np.mean(np.where((np.array(trQ_y)-np.array(trN_y))!=0,1,0))==1.0)

#define model
for k in [48]:#[12,24,36,48]:
    for r in [0.7]:#[0.3,0.5,0.7]:
        model = ATHNet(hash_size=k, type_size=4).cuda()#initialize model
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
        tl_loss  = TripletLoss(margin=r).cuda() #define TripletLoss 
        ce_loss  = nn.CrossEntropyLoss().cuda() #define ce mutli-classes

        #train model
        best_net, best_loss = None, float('inf')
        batchSize = 10
        for epoch in range(10):#iteration
            losses = []
            shuffled_idx = np.random.permutation(np.arange(len(trQ_sf)))
            train_q = trQ_sf[shuffled_idx]
            train_q_y = trQ_y[shuffled_idx]
            train_p = trP_sf[shuffled_idx]
            train_p_y = trP_y[shuffled_idx]
            train_n = trN_sf[shuffled_idx]
            train_n_y = trN_y[shuffled_idx]
            num_batches = len(trQ_sf) // batchSize
            for i in range(num_batches):
                optimizer.zero_grad()#grad vanish
                min_idx = i * batchSize
                max_idx = np.min([len(trQ_sf), (i+1)*batchSize])
                Q_batch = torch.from_numpy(train_q[min_idx:max_idx]).type(torch.FloatTensor).cuda()
                Q_y_batch = torch.from_numpy(train_q_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
                P_batch = torch.from_numpy(train_p[min_idx:max_idx]).type(torch.FloatTensor).cuda()
                P_y_batch = torch.from_numpy(train_p_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
                N_batch = torch.from_numpy(train_n[min_idx:max_idx]).type(torch.FloatTensor).cuda()
                N_y_batch = torch.from_numpy(train_n_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
                #forword
                Q_hash, Q_type = model(Q_batch.permute(0, 3, 1, 2))#permute the dims of matrix
                P_hash, P_type = model(P_batch.permute(0, 3, 1, 2))
                N_hash, N_type = model(N_batch.permute(0, 3, 1, 2))
                #loss
                hash_loss = tl_loss(Q_hash,P_hash,N_hash)
                type_loss = ce_loss(Q_type,Q_y_batch) + ce_loss(P_type,P_y_batch) + ce_loss(N_type,N_y_batch) #F.log_softmax+F.nll_loss
                loss = hash_loss+type_loss
                #backward
                loss.backward()
                #update parameters
                optimizer.step()
                #show loss
                sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
                sys.stdout.flush()     
                losses.append(loss.item())
            print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
            if np.mean(losses) < best_loss:
                best_loss = np.mean(losses)
                best_net = copy.deepcopy(model)
        print("best_loss = %.6f" % (best_loss))
        #release gpu memory
        model = model.cpu()
        tl_loss=tl_loss.cpu()
        ce_loss=ce_loss.cpu()
        torch.cuda.empty_cache()

        #hash code of train data from model
        batchSize = 10
        num_batches = len(trI) // batchSize
        trF = []
        for i in range(num_batches):
            min_idx = i * batchSize
            max_idx = np.min([len(trI), (i+1)*batchSize])
            I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
            X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
            I_batch = I_batch.cpu()
            X_batch = X_batch.cpu()
            torch.cuda.empty_cache()#release gpu memory
            trF.extend(X_batch.data.numpy().tolist())
            sys.stdout.write('\r {} / {} '.format(i, num_batches))
            sys.stdout.flush()

        #hash code of test data from model
        teY_pred = []
        teF = [] 
        num_batches = len(teY) // batchSize 
        for i in range(num_batches):
            min_idx = i * batchSize
            max_idx = np.min([len(teY), (i+1)*batchSize])
            x_batch = torch.from_numpy(np.array(teI[min_idx:max_idx])).type(torch.FloatTensor).cuda()
            x_hash, x_type = best_net(x_batch.permute(0, 3, 1, 2))#forword
            teF.extend(x_hash.cpu().data.numpy().tolist()) #record feature
            x_type = F.log_softmax(x_type,dim=1) 
            pred = x_type.max(1,keepdim=True)[1]
            teY_pred.extend(pred.cpu().data.numpy().tolist())
            sys.stdout.write('\r {} / {} '.format(i, num_batches))
            sys.stdout.flush()

        #performance of retrieval
        # buliding index of trainset
        tstart = time.time()
        cpu_index = faiss.IndexFlatL2(k) #
        gpu_index = faiss.index_cpu_to_all_gpus(cpu_index) #make all gpu usable
        gpu_index.add(np.ascontiguousarray(trF, dtype=np.float32)) #add data(must be float32) to index
        elapsed = time.time() - tstart    
        print('Completed buliding index in %d seconds' % int(elapsed))
        for topk in [10]: #[5,10,15,20]:
            MHR = [] #mean Hit ratio 
            MAP = [] #mean average precision
            MRR = [] #mean reciprocal rank
            scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=topk)
            for i, teVal in enumerate(teF):
                stype = teY[i]
                #perfromance
                pos_len = 0
                rank_len = 0
                mrr_flag = 0
                #for j in ranklist:
                for j in neighbors[i].tolist():
                    dtype = trY[j]
                    rank_len=rank_len+1
                    if stype==dtype:  #hit
                        MHR.append(1)
                        pos_len = pos_len +1
                        MAP.append(pos_len/rank_len) 
                        if mrr_flag==0: 
                            MRR.append(pos_len/rank_len)
                            mrr_flag =1
                    else: 
                        MHR.append(0)
                        MAP.append(0)   
            print("hash_size={}, margin={}, topk={}, mHR={:.6f}, mAP={:.6f}, mRR={:.6f}".format(k, r, topk, np.mean(MHR),np.mean(MAP),np.mean(MRR)))

 1000 / 1000 : loss = 5.250372Eopch:     1 mean_loss = 7.813445
 1000 / 1000 : loss = 2.081184Eopch:     2 mean_loss = 4.135663
 1000 / 1000 : loss = 2.827699Eopch:     3 mean_loss = 3.476321
 1000 / 1000 : loss = 0.034373Eopch:     4 mean_loss = 2.956136
 1000 / 1000 : loss = 0.888488Eopch:     5 mean_loss = 2.536000
 1000 / 1000 : loss = 2.151465Eopch:     6 mean_loss = 2.159403
 1000 / 1000 : loss = 6.509357Eopch:     7 mean_loss = 1.801342
 1000 / 1000 : loss = 0.586431Eopch:     8 mean_loss = 1.498455
 1000 / 1000 : loss = 2.998755Eopch:     9 mean_loss = 1.226462
 1000 / 1000 : loss = 0.403038Eopch:    10 mean_loss = 1.072182
best_loss = 1.072182
 1999 / 2000 Completed buliding index in 1 seconds
hash_size=48, margin=0.7, topk=10, mHR=0.776310, mAP=0.743370, mRR=0.911503


In [38]:
model = model.cpu()
tl_loss=tl_loss.cpu()
ce_loss=ce_loss.cpu()
torch.cuda.empty_cache()

In [42]:
#hash code of train data from model
tstart = time.time()
batchSize = 10
num_batches = len(trI) // batchSize
trF = []
for i in range(num_batches):
    min_idx = i * batchSize
    max_idx = np.min([len(trI), (i+1)*batchSize])
    I_batch = torch.from_numpy(np.array(trI[min_idx: max_idx])).type(torch.FloatTensor).cuda()
    X_batch,_ = best_net(I_batch.permute(0, 3, 1, 2))#forword
    I_batch = I_batch.cpu()
    X_batch = X_batch.cpu()
    torch.cuda.empty_cache()#release gpu memory
    trF.extend(X_batch.data.numpy().tolist())
    sys.stdout.write('\r {} / {} '.format(i, num_batches))
    sys.stdout.flush()
elapsed = time.time() - tstart   
print('Completed buliding index in %d seconds' % int(elapsed))    

 1999 / 2000 Completed buliding index in 28 seconds


In [47]:
tstart = time.time()
scores, neighbors = gpu_index.search(np.array(teF).astype('float32'), k=100)
elapsed = time.time() - tstart   
print(elapsed)
print('Completed buliding index in %d seconds' % int(elapsed))    

0.2575674057006836
Completed buliding index in 0 seconds


In [48]:
model = ATHNet(hash_size=k, type_size=4).cuda()#initialize model
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) #define optimizer
tl_loss  = TripletLoss(margin=r).cuda() #define TripletLoss 
ce_loss  = nn.CrossEntropyLoss().cuda() #define ce mutli-classes

#train model
best_net, best_loss = None, float('inf')
batchSize = 10
tstart = time.time()
for epoch in range(50):#iteration
    losses = []
    shuffled_idx = np.random.permutation(np.arange(len(trQ_sf)))
    train_q = trQ_sf[shuffled_idx]
    train_q_y = trQ_y[shuffled_idx]
    train_p = trP_sf[shuffled_idx]
    train_p_y = trP_y[shuffled_idx]
    train_n = trN_sf[shuffled_idx]
    train_n_y = trN_y[shuffled_idx]
    num_batches = len(trQ_sf) // batchSize
    for i in range(num_batches):
        optimizer.zero_grad()#grad vanish
        min_idx = i * batchSize
        max_idx = np.min([len(trQ_sf), (i+1)*batchSize])
        Q_batch = torch.from_numpy(train_q[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        Q_y_batch = torch.from_numpy(train_q_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        P_batch = torch.from_numpy(train_p[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        P_y_batch = torch.from_numpy(train_p_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        N_batch = torch.from_numpy(train_n[min_idx:max_idx]).type(torch.FloatTensor).cuda()
        N_y_batch = torch.from_numpy(train_n_y[min_idx:max_idx]).type(torch.LongTensor).cuda()
        #forword
        Q_hash, Q_type = model(Q_batch.permute(0, 3, 1, 2))#permute the dims of matrix
        P_hash, P_type = model(P_batch.permute(0, 3, 1, 2))
        N_hash, N_type = model(N_batch.permute(0, 3, 1, 2))
        #loss
        hash_loss = tl_loss(Q_hash,P_hash,N_hash)
        type_loss = ce_loss(Q_type,Q_y_batch) + ce_loss(P_type,P_y_batch) + ce_loss(N_type,N_y_batch) #F.log_softmax+F.nll_loss
        loss = hash_loss+type_loss
        #backward
        loss.backward()
        #update parameters
        optimizer.step()
        #show loss
        sys.stdout.write('\r {} / {} : loss = {}'.format(i+1, num_batches, float('%0.6f'%loss.item())))
        sys.stdout.flush()     
        losses.append(loss.item())
    print("Eopch: %5d mean_loss = %.6f" % (epoch + 1, np.mean(losses)))
    if np.mean(losses) < best_loss:
        best_loss = np.mean(losses)
        best_net = copy.deepcopy(model)
print("best_loss = %.6f" % (best_loss))
elapsed = time.time() - tstart   
print('Completed buliding index in %d seconds' % int(elapsed))  
#release gpu memory
model = model.cpu()
tl_loss=tl_loss.cpu()
ce_loss=ce_loss.cpu()
torch.cuda.empty_cache()

 1000 / 1000 : loss = 3.876657Eopch:     1 mean_loss = 8.415611
 1000 / 1000 : loss = 0.240579Eopch:     2 mean_loss = 4.064745
 1000 / 1000 : loss = 1.178148Eopch:     3 mean_loss = 3.322722
 1000 / 1000 : loss = 0.453971Eopch:     4 mean_loss = 2.540905
 1000 / 1000 : loss = 2.305858Eopch:     5 mean_loss = 2.019811
 1000 / 1000 : loss = 5.319075Eopch:     6 mean_loss = 1.553994
 1000 / 1000 : loss = 0.085987Eopch:     7 mean_loss = 1.314307
 1000 / 1000 : loss = 4.656636Eopch:     8 mean_loss = 0.949640
 1000 / 1000 : loss = 0.005121Eopch:     9 mean_loss = 0.927324
 1000 / 1000 : loss = 0.004219Eopch:    10 mean_loss = 0.593696
 1000 / 1000 : loss = 0.754326Eopch:    11 mean_loss = 0.574776
 1000 / 1000 : loss = 0.334976Eopch:    12 mean_loss = 0.542534
 1000 / 1000 : loss = 0.086265Eopch:    13 mean_loss = 0.428681
 1000 / 1000 : loss = 0.056847Eopch:    14 mean_loss = 0.420563
 1000 / 1000 : loss = 0.003638Eopch:    15 mean_loss = 0.298493
 1000 / 1000 : loss = 0.069464Eopch:    